In [5]:
%config InlineBackend.figure_format = 'retina'

In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import urllib3
urllib3.disable_warnings()
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import matplotlib.ticker
import matplotlib.transforms as transforms
import matplotlib.patches as mpatches

In [14]:
from influxdb_client import InfluxDBClient, Point
from influxdb_client.client.write.retry import WritesRetry
from influxdb_client.client.write_api import SYNCHRONOUS
import geocoder
import glob
from influxdb_client.domain.write_precision import WritePrecision

In [15]:
sns.set_style("whitegrid")
import matplotlib as mpl
matplotlib.rcParams['timezone'] = 'US/Eastern'
mpl.rcParams['figure.dpi'] = 150
mpl.rcParams['figure.figsize'] = [24, 12]
sns.set(font_scale=1.7)
sns.set_context("poster")

In [16]:
def query_from_influxdb(_url=None,
                        _token=None,
                        _org=None,
                        _id=None,
                        _bucket_name=None,
                        _columns_to_drop=None,
                        _measurement=None,
                        _field=None, ):
    
    """
        Function to query InfluxDB using influxdb-python-client library from https://github.com/influxdata/influxdb-client-python
            :param _measurement: Available options are flood-sensor-tidal-sensor-rain-gauge,weather,etc. Default is flood-sensor
            :param _field: Available options include batt_v and cloud_percent 
            :param _url: InfluxDB url
            :param _token: API token with read-access
            :param _org: InfluxDB organization name
            :param _id: Sensor/Deployment ID
            :param _bucket_name: Name of the bucket
            :param _columns_to_drop: Array-like column names to be dropped
        :returns a dataframe with all columns except dropped ones if any
    """
    result = []
    p = {"_id_value": _id,
         "_bucket_name": _bucket_name,
         "_measurement": _measurement, 
         "_field": _field, }


    with InfluxDBClient(url=_url, token=_token, org=_org, timeout=1000000) as client:
        query_api = client.query_api()
        if _measurement=="weather":  
            result = query_api.query_data_frame('''from(bucket: _bucket_name)
                                                  |> range(start: -20d, stop: now())
                                                  |> filter(fn: (r) => r["_measurement"] == _measurement)
                                                  |> filter(fn: (r) => r["_field"] == _field)
                                                  |> filter(fn: (r) => r["sensor_id"] == _id_value)
                                                  |> pivot(rowKey: ["_time"], columnKey: ["_field"], valueColumn: "_value")
                                                ''', params=p)   
        else:        
            result = query_api.query_data_frame('''from(bucket: _bucket_name)
                                                |> range(start: -20d)       
                                                |> filter(fn: (r) => r["_measurement"] == _measurement)
                                                |> filter(fn: (r) => r["deployment_id"] == _id_value)
                                                |> pivot(rowKey: ["_time"], columnKey: ["_field"], valueColumn: "_value")
                                                ''', params=p)

        if type(result) == list:
            result = pd.concat(result)

    result.drop(columns=['_start', '_stop', 'result', 'table'], inplace=True) #result
    result.rename(columns={'_time': 'time', '_measurement': 'measurement'}, inplace=True)
    result['time'] = pd.to_datetime(result['time'], format="%Y-%m-%d %H:%M:%S.%f", utc=True)
    result.set_index('time', inplace=True)
    result = result.sort_values(by=['time'])

    return result

In [17]:
sensor_ids = [ "owm-5717482", "owm-6318658176" ] #Carroll and 4th and what I assume to be Garfield/4th
dep_ids = [ 'weekly_poetic_guinea', 'daily_gentle_beetle' ] #Carroll and 4th and what I assume to be Garfield/4th

In [20]:
data = query_from_influxdb(_url='https://influxdb.floodlabs.nyc/',
                            _token='rvEpEpb9bDwrb1NbmlbnmT4HT_XCQxeKn3DHghJu9uABrm1qbHhR9D8eXsoMux9v073wttFRY6FLs6FxM7TuSw==',
                            _org='floodnet', 
                            _id='owm-5717482',
                            _bucket_name='floodnet-live',
                            _measurement="weather",
                            _field="cloud_percent",)

In [21]:
data.head()

,measurement,lat,lng,sensor_address_borough,sensor_address_country,sensor_address_neighbourhood,sensor_address_state,sensor_address_street,sensor_address_zip,sensor_id,sensor_name,sensor_types,weather_group,cloud_percent
time,,,,,,,,,,,,,,
2023-05-25 20:17:52+00:00,weather,40.5949,-74.0891,Richmond County,United States,Old Town Station,New York,Carmel Court,10304,owm-5717482,Old Town Station - openweathermap.org (owm-571...,"temp,humid,precip,pres,wind,solar",open_weather_map_forecast,20.0
2023-05-25 21:15:52+00:00,weather,40.5949,-74.0891,Richmond County,United States,Old Town Station,New York,Carmel Court,10304,owm-5717482,Old Town Station - openweathermap.org (owm-571...,"temp,humid,precip,pres,wind,solar",open_weather_map_forecast,20.0
2023-05-25 22:13:52+00:00,weather,40.5949,-74.0891,Richmond County,United States,Old Town Station,New York,Carmel Court,10304,owm-5717482,Old Town Station - openweathermap.org (owm-571...,"temp,humid,precip,pres,wind,solar",open_weather_map_forecast,40.0
2023-05-25 23:11:52+00:00,weather,40.5949,-74.0891,Richmond County,United States,Old Town Station,New York,Carmel Court,10304,owm-5717482,Old Town Station - openweathermap.org (owm-571...,"temp,humid,precip,pres,wind,solar",open_weather_map_forecast,40.0
2023-05-26 00:09:52+00:00,weather,40.5949,-74.0891,Richmond County,United States,Old Town Station,New York,Carmel Court,10304,owm-5717482,Old Town Station - openweathermap.org (owm-571...,"temp,humid,precip,pres,wind,solar",open_weather_map_forecast,40.0


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

data = query_from_influxdb(_url='https://influxdb.floodlabs.nyc/',
                            _token='rvEpEpb9bDwrb1NbmlbnmT4HT_XCQxeKn3DHghJu9uABrm1qbHhR9D8eXsoMux9v073wttFRY6FLs6FxM7TuSw==',
                            _org='floodnet', 
                            _id='owm-5717482',
                            _bucket_name='floodnet-live',
                            _measurement="weather",
                            _field="cloud_percent",)

data1= query_from_influxdb(_url='https://influxdb.floodlabs.nyc/',
                           _token='rvEpEpb9bDwrb1NbmlbnmT4HT_XCQxeKn3DHghJu9uABrm1qbHhR9D8eXsoMux9v073wttFRY6FLs6FxM7TuSw==',
                           _org='floodnet',
                           _id='weekly_poetic_guinea',
                           _bucket_name='floodnet-live',
                           _measurement="flood-sensor",
                           _field="batt_v",)

color = 'tab:red'
fig, ax1 = plt.subplots(figsize=(24,12))
data['cloud_percent'].plot(color=color)

ax1.set_xlabel('Past Twenty Days')
ax1.set_ylabel('Cloud Coverage (%)', color=color)
ax1.set_title('Carroll and 4th Ave')
ax1.tick_params(axis='y', labelcolor=color)

ax2 = ax1.twinx()  

color = 'tab:blue'
data1['batt_v'].plot(color=color)
ax2.set_ylabel('Battery Voltage (V)', color=color)  
ax2.tick_params(axis='y', labelcolor=color)

fig.tight_layout()
plt.show()


In [ ]:
#individual plot for just cloud coverage on Carroll and 4th Ave

data['cloud_percent'].plot(figsize=(24,12))
plt.xlabel('Last 20 Days')
plt.ylabel('Cloud Coverage (%)')
plt.title("Carroll and 4th Ave")


In [ ]:
#individual plot for just battery voltage level on Carroll and 4th Ave

data1['batt_v'].plot(figsize=(24,12), color='tab:red')
plt.xlabel('Last 20 Days')
plt.ylabel('Battery Voltage (V)')
plt.title("Carroll and 4th Ave")


In [ ]:
data2 = query_from_influxdb(_url='https://influxdb.floodlabs.nyc/',
                            _token='rvEpEpb9bDwrb1NbmlbnmT4HT_XCQxeKn3DHghJu9uABrm1qbHhR9D8eXsoMux9v073wttFRY6FLs6FxM7TuSw==',
                            _org='floodnet', 
                            _id='owm-6318658176',
                            _bucket_name='floodnet-live',
                            _measurement="weather",
                            _field="cloud_percent",)
    

data3= query_from_influxdb(_url='https://influxdb.floodlabs.nyc/',
                           _token='rvEpEpb9bDwrb1NbmlbnmT4HT_XCQxeKn3DHghJu9uABrm1qbHhR9D8eXsoMux9v073wttFRY6FLs6FxM7TuSw==',
                           _org='floodnet',
                           _id='weekly_poetic_guinea',
                          _bucket_name='floodnet-live',
                           _measurement="flood-sensor",
                           _field="batt_v",)

color = 'tab:red'
fig, ax1 = plt.subplots(figsize=(24,12))
data2['cloud_percent'].plot(color=color)

ax1.set_xlabel('Past Twenty Days')
ax1.set_ylabel('Cloud Coverage (%)', color=color)
ax1.set_title('Garfield and 4th Ave')
ax1.tick_params(axis='y', labelcolor=color)

ax2 = ax1.twinx()  

color = 'tab:blue'
data3['batt_v'].plot(color=color)
ax2.set_ylabel('Battery Voltage (V)', color=color)  
ax2.tick_params(axis='y', labelcolor=color)

fig.tight_layout()
plt.show()

In [ ]:
#individual plot for just cloud coverage on Garfield and 4th Ave

data2['cloud_percent'].plot(figsize=(24,12))
plt.xlabel('Last 20 Days')
plt.ylabel('Cloud Coverage (%)')
plt.title("Garfield and 4th Ave")


In [ ]:
#individual plot for just battery voltage level on Garfield and 4th Ave

data3['batt_v'].plot(figsize=(24,12), color='tab:red')
plt.xlabel('Last 20 Days')
plt.ylabel('Battery Voltage (V)')
plt.title("Garfield and 4th Ave")
